# Set Up

Run the following cell to load your data and some utility functions.

In [78]:
import pandas as pd
import numpy as np
from undyingkingdoms.static.metadata import *

Attempt to import races and write data to .xlsx

In [4]:
races = pd.DataFrame({"Races": metadata.metadata_races})
print(races)
# races.to_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Race Table')

    Races
0   Human
1     Elf
2   Dwarf
3  Goblin


Attempt to add modifiers to races.

In [5]:
modifiers = [mod[:-9] for mod in dir(metadata) if "modifier" in mod]
modifiers = pd.DataFrame({"Modifiers": modifiers})
# print(modifiers)

foo = {races.columns[0]: [], modifiers.columns[0]: []}
for race in races.Races:
    for mod in modifiers.Modifiers:
        foo[races.columns[0]].append(race)
        foo[modifiers.columns[0]].append(mod)

race_mods = pd.DataFrame(foo)
race_mods.to_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Race Modifiers')

Attempt to pull modifier values from metadata.

In [164]:
modifiers = [mod for mod in dir(metadata) if "modifier" in mod]
mod_dfs = {' '.join(mod.split('_')).title(): pd.DataFrame(getattr(metadata, mod)) for mod in modifiers}

mod_types = [' '.join(mod.split('_')).title() for mod in modifiers]
cols = ["Race", "Background", "Modifier Label", "Modifier Type", "Modifier Value", "Modifier Description"]
# cols = ['Races', 'BackgroundsModifier Names', 'Amount Of Thieves Modifier', 'Birth Rate Modifier', 'Death Rate Modifier', 'Defense Per Citizen Modifier', 'Food Consumed Modifier', 'Happiness Modifier', 'Income Modifier', 'Infiltration Success Modifier', 'Offensive Power Modifier', 'Production Per Worker Modifier']
df = pd.DataFrame(index=range(12), columns=cols)

i = 0
# print(cols)
for mod_key, mod_df in mod_dfs.items():
    for r_b in mod_df:
        if r_b in metadata.metadata_races:
            df.Race[i] = r_b
        else:  # col is Background
            df.Background[i] = r_b
        df['Modifier Type'][i] = mod_key
        df['Modifier Label'][i] = mod_df[r_b][0]
        df['Modifier Value'][i] = mod_df[r_b][1]
        i += 1

for race in metadata.metadata_races:
    if race not in df.Race.dropna().unique():
#         print(race)
        df.Race[i] = race
        i += 1

for background in metadata.metadata_backgrounds:
    if background not in df.Background.dropna().unique():
        print(background)
        df.Background[i] = background
        i += 1
df = df.sort_values(['Race', 'Background', 'Modifier Label']).reset_index(drop=True)


for name in df['Modifier Label']:
    try:
        row = df['Modifier Label'] == name
        val = df.loc[row, "Modifier Value"].iloc[0]
        if int(val) == val:  # is a fixed value
            if val > 0:
                val = '+' + str(int(val))
        else:  # is a percent
            if val > 0:
                val = '+' + str(int(val * 100))
            val = str(val) + '%'
        label = df.loc[row, "Modifier Type"].iloc[0][:-9]
        df.loc[row, 'Modifier Description'] = '{}: {} {}'.format(name, val, label)
    except IndexError:
        pass

df.to_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Modifiers')
df

,Race,Background,Modifier Label,Modifier Type,Modifier Value,Modifier Description
0,Dwarf,NaN,Dwarven Made,Production Per Worker Modifier,0.15,Dwarven Made: +15% Production Per Worker
1,Dwarf,NaN,Ravenous,Food Consumed Modifier,0.15,Ravenous: +15% Food Consumed
2,Elf,NaN,Citizen Militia,Defense Per Citizen Modifier,1,Citizen Militia: +1 Defense Per Citizen
3,Elf,NaN,Elders,Birth Rate Modifier,-0.15,Elders: -0.15% Birth Rate
4,Goblin,NaN,Expendable,Birth Rate Modifier,0.1,Expendable: +10% Birth Rate
5,Goblin,NaN,Infighting,Happiness Modifier,-2,Infighting: -2 Happiness
6,Human,NaN,NaN,NaN,NaN,NaN
7,NaN,Engineer,Craftsman,Production Per Worker Modifier,0.2,Craftsman: +20% Production Per Worker
8,NaN,Merchant,Silver Tongue,Income Modifier,0.15,Silver Tongue: +15% Income
9,NaN,Rogue,Army of Shadows,Amount Of Thieves Modifier,2,Army of Shadows: +2 Amount Of Thieves


Try and work out what data to pull into the Guide Race Compare page.

In [197]:
df = pd.read_excel('undyingkingdoms/static/metadata/all.xlsx', sheet_name='Modifiers')

for race in df.Race.unique():
    select = df['Race'] == race
    for row in df.loc[select, 'Modifier Description']:
        if row:
            print(row)

Dwarven Made: +15% Production Per Worker
Ravenous: +15% Food Consumed
Citizen Militia: +1 Defense Per Citizen
Elders: -0.15% Birth Rate
Expendable: +10% Birth Rate
Infighting: -2 Happiness
nan


Attempt to pull soldier data out of metadata files.